In [2]:
from resnet import ResNet18
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os

In [11]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [12]:
net = ResNet18()

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [14]:
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9) #, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [16]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

In [17]:
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total

In [18]:
for epoch in range(200):
    train(epoch)
    test(epoch)
    scheduler.step()


Epoch: 0
390 391 Loss: 1.672 | Acc: 39.164% (19582/50000)
99 100 Loss: 2.376 | Acc: 34.060% (3406/10000)

Epoch: 1
390 391 Loss: 1.101 | Acc: 60.186% (30093/50000)
99 100 Loss: 1.336 | Acc: 56.330% (5633/10000)

Epoch: 2
390 391 Loss: 0.840 | Acc: 70.248% (35124/50000)
99 100 Loss: 0.835 | Acc: 71.870% (7187/10000)

Epoch: 3
390 391 Loss: 0.680 | Acc: 76.326% (38163/50000)
99 100 Loss: 0.762 | Acc: 74.270% (7427/10000)

Epoch: 4
390 391 Loss: 0.588 | Acc: 79.534% (39767/50000)
99 100 Loss: 0.696 | Acc: 77.430% (7743/10000)

Epoch: 5
390 391 Loss: 0.516 | Acc: 82.148% (41074/50000)
99 100 Loss: 0.520 | Acc: 82.650% (8265/10000)

Epoch: 6
390 391 Loss: 0.467 | Acc: 83.938% (41969/50000)
99 100 Loss: 0.551 | Acc: 82.140% (8214/10000)

Epoch: 7
390 391 Loss: 0.424 | Acc: 85.292% (42646/50000)
99 100 Loss: 0.843 | Acc: 73.530% (7353/10000)

Epoch: 8
390 391 Loss: 0.387 | Acc: 86.678% (43339/50000)
99 100 Loss: 0.541 | Acc: 82.530% (8253/10000)

Epoch: 9
390 391 Loss: 0.356 | Acc: 87.650% (

In [ ]:
if acc > best_acc:
    print('Saving..')
    state = {
        'net': net.state_dict(),
        'acc': acc,
        'epoch': epoch,
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/ckpt.pth')
    best_acc = acc